In [1]:
from pyjarowinkler import distance
from math import*
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from sklearn.metrics import euclidean_distances
import gensim
import pymysql
import pymysql.cursors
from gensim import corpora, models
import math
from textblob import TextBlob as tb
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import requests
import json
import time
import itertools

In [2]:
def connect_to_database():
    options = {
        'user': "root",
        'passwd': "root",
        'db': "KnowBase",
        'cursorclass' : pymysql.cursors.DictCursor
    }
    db = pymysql.connect(**options)
    db.autocommit(True)
    return db

db = connect_to_database()
cur = db.cursor()
string = "select b.freebase_id, b.entity, b.relation, b.value, b.link_am_score, b.link_scroe, b.base_id, "
string += "b.freebase_entity from NoiseEntity b"
cur.execute(string)
results = cur.fetchall()
groundTrue = {}
label = []
entity = []
entityDict = {}
y = []
i= 0
wordList = []
file_t = open("data.txt", 'w')
for result in results:
    if result["freebase_id"] not in groundTrue:
        groundTrue[result["freebase_id"]] = i
        label.append(result["freebase_entity"].lower())
        i = i + 1
    if result["entity"] not in entityDict:
        entityDict[result["entity"].lower()] = 1
    entity.append(result["entity"].lower())
    y.append(groundTrue[result["freebase_id"]])
    
    temp = {}
    temp["n"] = result["entity"].lower().split()
    list_A = result["relation"].lower().split()
    list_A.extend(result["value"].lower().split())
    temp["A"] = list_A
    temp["f_id"] = result["freebase_id"]
    temp["f_entity"] = result["freebase_entity"]
    temp["score"] = result["link_scroe"]
    temp["am_score"] = result["link_am_score"]
    temp["relation"] = result["relation"]
    temp["value"] = result["value"]
    temp["id"] = result["base_id"]
    wordList.append(temp)

In [3]:
def macro_precision(result, y, n_cluster):
    groupDict = {}
    i= 0
    for item in result:
        if item in groupDict:
            groupDict[item].append(i)
        else:
            groupDict[item] = [i]
        i = i + 1

    trueDict = {}
    i = 0
    for item in y:
        if item not in trueDict:
            trueDict[item] = [i]
        else:
            trueDict[item].append(i)
        i = i + 1

    inverseTrue = {}
    for key in trueDict.keys():
        for item in trueDict[key]:
            inverseTrue[item] = key
    
    totalNotSame = 0
    for key in groupDict.keys():
        temp = groupDict[key]
        f_id = -1
        i = 0
        control = True
        while i < len(temp) and control == True:
            f_id_cur = inverseTrue[temp[i]]
            if f_id == -1:
                f_id = f_id_cur
            else:
                if f_id_cur != f_id:
                    totalNotSame += 1
                    control = False
            i += 1
                    
    return (n_cluster-totalNotSame) / n_cluster

def macro_recall(result, y, n_cluster):
    return macro_precision(y, result, n_cluster)
    

def micro_precision(result, y):
    lenC = len(result)
    groupDict = {}
    i= 0
    for item in result:
        if item in groupDict:
            groupDict[item].append(i)
        else:
            groupDict[item] = [i]
        i = i + 1

    trueDict = {}
    i = 0
    for item in y:
        if item not in trueDict:
            trueDict[item] = [i]
        else:
            trueDict[item].append(i)
        i = i + 1

    inverseTrue = {}
    for key in trueDict.keys():
        for item in trueDict[key]:
            inverseTrue[item] = key
    
    totalSame = 0
    for key in groupDict.keys():
        temp = groupDict[key]
        output = {}
        maxO = 0
        maxKey = ""
        for item in temp:
            f_id = inverseTrue[item]
            if f_id not in output:
                output[f_id] = 1
            else:
                output[f_id] += 1
            
            if output[f_id] > maxO:
                maxO = output[f_id]
                maxKey = key
                
        totalSame += maxO
    return  totalSame / lenC
                
def micro_recall(result, y):
    return micro_precision(y, result)


def pairwise_precision(result, y):
    lenC = len(result)
    groupDict = {}
    i= 0
    for item in y:
        if item in groupDict:
            groupDict[item].append(i)
        else:
            groupDict[item] = [i]
        i = i + 1

    trueDict = {}
    i = 0
    for item in result:
        if item not in trueDict:
            trueDict[item] = [i]
        else:
            trueDict[item].append(i)
        i = i + 1

    inverseTrue = {}
    for key in trueDict.keys():
        for item in trueDict[key]:
            inverseTrue[item] = key
    
    totalPair = 0
    totalHit = 0
    for key in groupDict.keys():
        temp = groupDict[key]
        i = 0
        j = 0
        while i < len(temp):
            while j < len(temp):
                f_id_i = inverseTrue[temp[i]]
                f_id_j = inverseTrue[temp[j]]
                if f_id_i == f_id_j:
                    totalHit += 1
                j += 1
                totalPair += 1
            i += 1
        
    return  totalHit / totalPair


def pairwise_recall(result, y):
    lenC = len(result)
    groupDict = {}
    i= 0
    for item in y:
        if item in groupDict:
            groupDict[item].append(i)
        else:
            groupDict[item] = [i]
        i = i + 1

    trueDict = {}
    i = 0
    for item in result:
        if item not in trueDict:
            trueDict[item] = [i]
        else:
            trueDict[item].append(i)
        i = i + 1

    inverseGroup = {}
    for key in groupDict.keys():
        for item in groupDict[key]:
            inverseGroup[item] = key
    
    totalPair = 0
    totalHit = 0
    for key in trueDict.keys():
        temp = trueDict[key]
        i = 0
        j = 0
        while i < len(temp):
            while j < len(temp):
                f_id_i = inverseGroup[temp[i]]
                f_id_j = inverseGroup[temp[j]]
                if f_id_i == f_id_j:
                    totalHit += 1
                j += 1
                totalPair += 1
            i += 1
        
    return  totalHit / totalPair

In [4]:
def string_identity(m1, m2):
    entity1 = m1["n"]
    entity2 = m2["n"]
    str1 = " ".join(entity1)
    str2 = " ".join(entity2)
    if str1 == str2:
        return 1
    else:
        return 0

## suppose we say that cosine+average and string similarity produce the best result
## to find number cluster, we want to compare their result
## first, use cosine+average to predict and assume that this is the "groundtrue"
## we then test the accuracy of string similarity base on this "groundtrue"
def apply_string_identity(wordList, n_cluster_in):
    out = []
    for item1 in wordList:
        temp = []
        for item2 in wordList:
            result = np.exp(-string_identity(item1, item2))
            temp.append(result)
        out.append(temp)
    model = AgglomerativeClustering(n_clusters=n_cluster_in, affinity="precomputed", linkage="average")
    model.fit(out)
    result = model.labels_
    Out_to_print = {}
    return result

def applyHAC(n_cluster_in, affinity_in, linkage_in):
    model = AgglomerativeClustering(n_clusters=n_cluster_in, affinity=affinity_in, linkage=linkage_in)
    model.fit(entity_input)
    result = model.labels_
    return result

In [5]:
bagList = []
bagDict = {}
i = 0
for key in entityDict.keys():
    for item in key.split():
        if item not in bagDict:
            bagList.append(item.lower())
            bagDict[item.lower()] = i
            i = i + 1
            
entity_input = []
temp = [0 for x in range(i)]

for word in entity:
    temp = [0 for x in range(i)]
    for item in word.split():
        index = bagDict[item.lower()]
        temp[index] = 1
    entity_input.append(temp) 

In [14]:
clus = 130
while clus <= 160:
    fakeGroundTrue = applyHAC(clus, "cosine", "average")
    realOut = applyHAC(clus, "euclidean", "ward")
    ## changing the order of realout and fakeGroundTrue is just switching recall and precision    
    print("one round: " + str(clus))
    print(macro_precision(realOut, fakeGroundTrue, clus))
    print(macro_recall(realOut, fakeGroundTrue, clus))
    ## don't need to look at micro since the more cluster you have, the more freedom it has and micro precision
    ## should increase, same as pairwise, hit increase as freedom increase
    ## print(micro_precision(realOut, fakeGroundTrue))
    ## print(micro_recall(realOut, fakeGroundTrue))
    ##print(pairwise_precision(realOut, fakeGroundTrue))
    ##print(pairwise_recall(realOut, fakeGroundTrue))
    clus += 5

one round: 130
0.9923076923076923
0.9230769230769231
one round: 135
0.9925925925925926
0.9259259259259259
one round: 140
0.9928571428571429
0.9071428571428571
one round: 145
0.993103448275862
0.903448275862069
one round: 150
0.9933333333333333
0.9
one round: 155
0.9870967741935484
0.9096774193548387
one round: 160
0.9875
0.9375


In [15]:
clus = 130
while clus <= 160:
    realOut = applyHAC(clus, "cosine", "average")
    fakeGroundTrue = applyHAC(clus, "euclidean", "ward")
    print("one round: " + str(clus))
    print(macro_precision(realOut, fakeGroundTrue, clus))
    print(macro_recall(realOut, fakeGroundTrue, clus))
    clus += 5

one round: 130
0.9230769230769231
0.9923076923076923
one round: 135
0.9259259259259259
0.9925925925925926
one round: 140
0.9071428571428571
0.9928571428571429
one round: 145
0.903448275862069
0.993103448275862
one round: 150
0.9
0.9933333333333333
one round: 155
0.9096774193548387
0.9870967741935484
one round: 160
0.9375
0.9875


In [20]:
clus = 130
while clus <= 160:
    fakeGroundTrue = applyHAC(clus, "cosine", "average")
    realOut = apply_string_identity(wordList, clus)
    print("one round: " + str(clus))
    print(macro_precision(realOut, fakeGroundTrue, clus))
    print(macro_recall(realOut, fakeGroundTrue, clus))
    ## don't need to look at micro since the more cluster you have, the more freedom it has and micro precision
    ## should increase, same as pairwise, hit increase as freedom increase
    ## print(micro_precision(realOut, fakeGroundTrue))
    ## print(micro_recall(realOut, fakeGroundTrue))
    ##print(pairwise_precision(realOut, fakeGroundTrue))
    ##print(pairwise_recall(realOut, fakeGroundTrue))
    clus += 5

one round: 130
0.9923076923076923
0.7923076923076923
one round: 135
0.9925925925925926
0.762962962962963
one round: 140
0.9928571428571429
0.7357142857142858
one round: 145
0.993103448275862
0.696551724137931
one round: 150
0.9933333333333333
0.6866666666666666
one round: 155
0.9935483870967742
0.6903225806451613
one round: 160
0.99375
0.71875


In [21]:
def string_similarity(m1, m2):
    entity1 = m1["n"]
    entity2 = m2["n"]
    str1 = " ".join(entity1)
    str2 = " ".join(entity2)
    return distance.get_jaro_distance(str1, str2, winkler=True, scaling=0.1)

def jaccard_similarity(x, y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def string_similarity_matrix(wordList):
    out = []
    for item1 in wordList:
        temp = []
        for item2 in wordList:
            result = string_similarity(item1, item2)
            if result < 0.8:
                result = 0
            result = np.exp(-result)
            temp.append(result)
        out.append(temp)
    return out

def apply_string_similarity(n_cluster_in, out):
    model = AgglomerativeClustering(n_clusters=n_cluster_in, affinity="precomputed", linkage="average")
    model.fit(out)
    result = model.labels_
    return result

In [22]:
clus = 130
out = string_similarity_matrix(wordList)
while clus <= 160:
    fakeGroundTrue = applyHAC(clus, "cosine", "average")
    realOut = apply_string_similarity(clus, out)
    print("one round: " + str(clus))
    print(macro_precision(realOut, fakeGroundTrue, clus))
    print(macro_recall(realOut, fakeGroundTrue, clus))
    clus += 5

one round: 130
0.8923076923076924
0.823076923076923
one round: 135
0.8888888888888888
0.7925925925925926
one round: 140
0.9
0.7571428571428571
one round: 145
0.896551724137931
0.7310344827586207
one round: 150
0.86
0.7266666666666667
one round: 155
0.8258064516129032
0.7354838709677419
one round: 160
0.8125
0.7625


In [54]:
out = []
for item1 in wordList:
    temp = []
    for item2 in wordList:
        result = np.exp(-string_identity(item1, item2))
        temp.append(result)
    out.append(temp)
clustering = AgglomerativeClustering()
clustering.fit(out)
[{'node_id': next(itertools.count(X.shape[0])), 'left': x[0], 'right':x[1]} for x in clustering.children_]

[{'left': 0, 'node_id': 750, 'right': 1},
 {'left': 8, 'node_id': 750, 'right': 6918},
 {'left': 2, 'node_id': 750, 'right': 45},
 {'left': 3, 'node_id': 750, 'right': 5},
 {'left': 4, 'node_id': 750, 'right': 7},
 {'left': 6, 'node_id': 750, 'right': 6921},
 {'left': 9, 'node_id': 750, 'right': 6923},
 {'left': 10, 'node_id': 750, 'right': 6922},
 {'left': 21, 'node_id': 750, 'right': 6919},
 {'left': 12, 'node_id': 750, 'right': 6924},
 {'left': 11, 'node_id': 750, 'right': 6925},
 {'left': 17, 'node_id': 750, 'right': 6928},
 {'left': 13, 'node_id': 750, 'right': 6927},
 {'left': 16, 'node_id': 750, 'right': 6930},
 {'left': 14, 'node_id': 750, 'right': 15},
 {'left': 26, 'node_id': 750, 'right': 6932},
 {'left': 25, 'node_id': 750, 'right': 6931},
 {'left': 23, 'node_id': 750, 'right': 6929},
 {'left': 18, 'node_id': 750, 'right': 19},
 {'left': 29, 'node_id': 750, 'right': 6936},
 {'left': 20, 'node_id': 750, 'right': 22},
 {'left': 27, 'node_id': 750, 'right': 6926},
 {'left': 44